In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle

In [2]:
# train = pd.read_pickle(FILE.train_lgb_base.value)
train = pd.read_pickle('../../data/lgb_feature/train_lgb_agg2.pickle')
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(FILE.test_lgb_base.value)
test = pd.read_pickle('../../data/lgb_feature/test_lgb_agg2.pickle')
print('test shape is: {}'.format(test.shape))
test['click'] = np.nan

train shape is: (1001650, 74)
test shape is: (40024, 74)


In [3]:
ignore_col = ['instance_id','click','time']
search_feature = list(set(train.columns) - set(ignore_col))

# LGBM

In [4]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=12,
            n_estimators=30000,
            learning_rate=lr,
#             max_bin =10000,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_final_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=41):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [5]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [1001]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.1]
    nfolds = 7
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [6]:
# split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)

In [7]:
def drop_each_col(train_df,holdout,test,search_feature,target,report_path='report_drop.csv',by='all'):
    """
    train_df, the train set which does not has holdout
    by = 'cv','ho', other == cv+ ho
    """
    #1 run base line
    need_caculate_base_line = True
    try:
        report = pd.read_csv(report_path)
        print('found report! resume from last time!')
    except:
        print('calculate importance to generate report...')
        split_all, gain_all,holdout_base,holdout_mean,cv_base,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)
        split_all = split_all.sort_values('importance')
        fea = [f for f in split_all.feature.values if f in search_feature]
        report = pd.DataFrame({'feature':fea})
        report['order'] = report.index
        report['holdout'] = np.nan
        report['holdout_mean'] = np.nan
        report['search'] = 'n'
        report['drop'] = 'n'
        report['cv'] = np.nan
        need_caculate_base_line = False
    droped = report[report['drop']=='y'].copy()
    droped_col = []
    if len(droped) == 0:
        if need_caculate_base_line:
            print('calculate base line...')
            _, _,holdout_base,_,cv_base,_,_,_ = runlgb(train_df, 
                                                       test, 
                                                       holdout)
    else:
        droped = droped.sort_values('order')
        holdout_base = droped.iloc[-1]['holdout']
        cv_base = droped.iloc[-1]['cv']
        droped_col = list(droped['feature'].values)
    print('---- holdout base is {}. cv base is {}'.format(holdout_base,cv_base))
    print('dropped features are: {}'.format(droped_col))
    best_col = list(set(train_df.columns) - set(droped_col))

    searched = report[report['search']=='y'].copy()
    start_index = len(searched)


    for i in range(start_index,len(report)):
        col = report.iloc[i]['feature']
        print('start... {}/{} fe:{}'.format(i,len(report),col))
        cur_col = list(set(best_col) - set([col]))
        _, _,ho_cur,ho_mean_cur,cv_cur,_,_,_ = runlgb(train_df[cur_col], 
                                                   test[cur_col], 
                                                   holdout[cur_col])
        report.loc[i,'holdout'] = ho_cur
        report.loc[i,'holdout_mean'] = ho_mean_cur
        report.loc[i,'search'] = 'y'
        report.loc[i,'cv'] = cv_cur
        if by == 'cv' and cv_cur < cv_base:
            cv_base = cv_cur
            report.loc[i,'drop'] = 'y'
            print('#### {}/{} YES DROP col: {}'.format(i,len(report),col))
            best_col = list(set(best_col) - set([col]))
        elif by == 'ho' and ho_cur < holdout_base:
            holdout_base = ho_cur
            report.loc[i,'drop'] = 'y'
            print('#### {}/{} YES DROP col: {}'.format(i,len(report),col))
            best_col = list(set(best_col) - set([col]))
        elif cv_cur < cv_base and ho_cur < holdout_base:
            cv_base = cv_cur
            holdout_base = ho_cur
            report.loc[i,'drop'] = 'y'
            print('#### {}/{} YES DROP col: {}'.format(i,len(report),col))
            best_col = list(set(best_col) - set([col]))
        else:
            report.loc[i,'drop'] = 'n'
            print('#### {}/{} NO DROP col: {}'.format(i,len(report),col))
        report.to_csv(report_path,index=False)
        print('')

In [8]:
report_path = 'report_drop_agg2_cv.csv'
target='click'
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_df = train.loc[train_index].copy()
holdout = train.loc[holdout_index].copy()
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train_df[target].mean(),holdout[target].mean()))

drop_each_col(train_df,holdout,test,search_feature,target,report_path=report_path,by='cv')

train shape: (961626, 74), holdout shape: (40024, 74)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
found report! resume from last time!
calculate base line...
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 74) (40024, 74) (40024, 74)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  363] AUC : ho: 0.426930 / te: 0.415778 / tr: 0.406033 (diff: 0.011152)
Fold  2 [  487] AUC : ho: 0.426494 / te: 0.415308 / tr: 0.402764 (diff: 0.011186)
Fold  3 [  537] AUC : ho: 0.426661 / te: 0.414947 / tr: 0.401499 (diff: 0.011714)
Fold  4 [  566] AUC : ho: 0.426922 / te: 0.415423 / tr: 0.400466 (diff: 0.011499)
Fold  5 [  556] AUC : ho: 0.426476 / te: 0.416498 / tr: 0.400716 (diff: 0.009978)
Fold  6 [  493] AUC : ho: 0.427278 / te: 0.415962 / tr: 0.402571 (diff: 0.011316)
Fold  7 [  356] AUC : ho: 0.427081 / te: 0.415697 / tr: 0.406123 (diff: 0.011383

Fold  1 [  333] AUC : ho: 0.426550 / te: 0.415843 / tr: 0.406858 (diff: 0.010707)
Fold  2 [  366] AUC : ho: 0.426505 / te: 0.415341 / tr: 0.405969 (diff: 0.011164)
Fold  3 [  606] AUC : ho: 0.426624 / te: 0.415105 / tr: 0.399937 (diff: 0.011519)
Fold  4 [  576] AUC : ho: 0.427050 / te: 0.415506 / tr: 0.400313 (diff: 0.011543)
Fold  5 [  466] AUC : ho: 0.427085 / te: 0.416594 / tr: 0.403092 (diff: 0.010491)
Fold  6 [  588] AUC : ho: 0.426998 / te: 0.415881 / tr: 0.400002 (diff: 0.011118)
Fold  6 [  493] AUC : ho: 0.427281 / te: 0.415962 / tr: 0.402571 (diff: 0.011319)
Fold  7 [  356] AUC : ho: 0.427083 / te: 0.415695 / tr: 0.406123 (diff: 0.011388)
Full HO score 0.425915
FULL HO mean 0.426848, std 0.000274
FULL TE mean 0.415695, std 0.000412
FULL TR mean 0.403218, std 0.002112
FULL DIFF mean 0.011153, std 0.000494
Run LightGBM with kfold - done in 139s
#### 9/71 NO DROP col: count_has_usertags

start... 10/71 fe:count_os
#############################################
0.07 1001 5 16 20 1 

Fold  6 [  390] AUC : ho: 0.426711 / te: 0.416123 / tr: 0.405277 (diff: 0.010588)
Fold  7 [  430] AUC : ho: 0.427147 / te: 0.415641 / tr: 0.403932 (diff: 0.011506)
Full HO score 0.425977
FULL HO mean 0.426908, std 0.000200
FULL TE mean 0.415718, std 0.000466
FULL TR mean 0.402924, std 0.001871
FULL DIFF mean 0.011190, std 0.000515
Run LightGBM with kfold - done in 141s
#### 16/71 NO DROP col: count_advert_name

start... 17/71 fe:count_devtype
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  369] AUC : ho: 0.426811 / te: 0.415867 / tr: 0.405711 (diff: 0.010944)
Fold  2 [  511] AUC : ho: 0.426322 / te: 0.415190 / tr: 0.402207 (diff: 0.011132)
Fold  3 [  610] AUC : ho: 0.427068 / te: 0.415113 / tr: 0.399694 (diff: 0.011955)
Fold  4 [  483] AUC : ho: 0.426847 / te: 0.415484 / tr: 0.40

#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  468] AUC : ho: 0.426795 / te: 0.415778 / tr: 0.403313 (diff: 0.011017)
Fold  2 [  492] AUC : ho: 0.426598 / te: 0.415335 / tr: 0.402721 (diff: 0.011263)
Fold  3 [  502] AUC : ho: 0.426857 / te: 0.415303 / tr: 0.402601 (diff: 0.011554)
Fold  4 [  545] AUC : ho: 0.427141 / te: 0.415609 / tr: 0.401212 (diff: 0.011532)
Fold  5 [  465] AUC : ho: 0.427394 / te: 0.416772 / tr: 0.403172 (diff: 0.010622)
Fold  6 [  410] AUC : ho: 0.427029 / te: 0.416032 / tr: 0.404777 (diff: 0.010996)
Fold  7 [  555] AUC : ho: 0.426745 / te: 0.415584 / tr: 0.401008 (diff: 0.011160)
Full HO score 0.425969
FULL HO mean 0.426937, std 0.000250
FULL TE mean 0.415773, std 0.000469
FULL TR mean 0.402686, std 0.001195
FULL DIFF mean 0.011164, std 0.000303
Run LightG

Fold  4 [  329] AUC : ho: 0.426962 / te: 0.415441 / tr: 0.406848 (diff: 0.011521)
Fold  5 [  459] AUC : ho: 0.426773 / te: 0.416641 / tr: 0.403240 (diff: 0.010131)
Fold  6 [  507] AUC : ho: 0.426611 / te: 0.415929 / tr: 0.401980 (diff: 0.010682)
Fold  7 [  366] AUC : ho: 0.426908 / te: 0.415792 / tr: 0.405741 (diff: 0.011116)
Full HO score 0.425882
FULL HO mean 0.426748, std 0.000332
FULL TE mean 0.415694, std 0.000490
FULL TR mean 0.403946, std 0.002544
FULL DIFF mean 0.011054, std 0.000639
Run LightGBM with kfold - done in 131s
#### 31/71 NO DROP col: count_creative_area

start... 32/71 fe:count_campaign_id
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  437] AUC : ho: 0.426743 / te: 0.415816 / tr: 0.404026 (diff: 0.010928)
Fold  2 [  273] AUC : ho: 0.427142 / te: 0.415384 / tr

#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  302] AUC : ho: 0.426880 / te: 0.415934 / tr: 0.407709 (diff: 0.010946)
Fold  2 [  375] AUC : ho: 0.426585 / te: 0.415303 / tr: 0.405674 (diff: 0.011282)
Fold  3 [  370] AUC : ho: 0.426490 / te: 0.415093 / tr: 0.405768 (diff: 0.011398)
Fold  4 [  481] AUC : ho: 0.426376 / te: 0.415329 / tr: 0.402796 (diff: 0.011047)
Fold  5 [  604] AUC : ho: 0.426765 / te: 0.416535 / tr: 0.399679 (diff: 0.010230)
Fold  6 [  444] AUC : ho: 0.426828 / te: 0.416051 / tr: 0.403783 (diff: 0.010777)
Fold  7 [  536] AUC : ho: 0.426672 / te: 0.415567 / tr: 0.401371 (diff: 0.011105)
Full HO score 0.425765
FULL HO mean 0.426657, std 0.000170
FULL TE mean 0.415688, std 0.000472
FULL TR mean 0.403826, std 0.002579
FULL DIFF mean 0.010969, std 0.000357
Run LightG

Fold  5 [  562] AUC : ho: 0.427172 / te: 0.416879 / tr: 0.400670 (diff: 0.010293)
Fold  6 [  405] AUC : ho: 0.426947 / te: 0.416131 / tr: 0.404771 (diff: 0.010816)
Fold  7 [  532] AUC : ho: 0.426700 / te: 0.415624 / tr: 0.401578 (diff: 0.011076)
Full HO score 0.425998
FULL HO mean 0.426863, std 0.000191
FULL TE mean 0.415818, std 0.000524
FULL TR mean 0.403603, std 0.001615
FULL DIFF mean 0.011045, std 0.000375
Run LightGBM with kfold - done in 140s
#### 46/71 NO DROP col: count_app_id

start... 47/71 fe:count_inner_slot_id
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  264] AUC : ho: 0.427131 / te: 0.416000 / tr: 0.408819 (diff: 0.011131)
Fold  2 [  409] AUC : ho: 0.426822 / te: 0.415311 / tr: 0.404912 (diff: 0.011511)
Fold  3 [  547] AUC : ho: 0.426836 / te: 0.415241 / tr: 0.4

Fold  1 [  364] AUC : ho: 0.426977 / te: 0.415851 / tr: 0.406070 (diff: 0.011126)
Fold  2 [  432] AUC : ho: 0.426859 / te: 0.415494 / tr: 0.404262 (diff: 0.011365)
Fold  3 [  501] AUC : ho: 0.426870 / te: 0.415185 / tr: 0.402505 (diff: 0.011685)
Fold  4 [  364] AUC : ho: 0.426474 / te: 0.415305 / tr: 0.405872 (diff: 0.011169)
Fold  5 [  592] AUC : ho: 0.427002 / te: 0.416771 / tr: 0.400131 (diff: 0.010231)
Fold  6 [  471] AUC : ho: 0.426704 / te: 0.416176 / tr: 0.403106 (diff: 0.010528)
Fold  7 [  349] AUC : ho: 0.427090 / te: 0.415764 / tr: 0.406434 (diff: 0.011326)
Full HO score 0.425977
FULL HO mean 0.426854, std 0.000193
FULL TE mean 0.415792, std 0.000508
FULL TR mean 0.404054, std 0.002130
FULL DIFF mean 0.011062, std 0.000469
Run LightGBM with kfold - done in 134s
#### 54/71 NO DROP col: make

start... 55/71 fe:app_id
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 

#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  432] AUC : ho: 0.426539 / te: 0.415661 / tr: 0.404272 (diff: 0.010877)
Fold  2 [  440] AUC : ho: 0.426568 / te: 0.415237 / tr: 0.403995 (diff: 0.011331)
Fold  3 [  545] AUC : ho: 0.426856 / te: 0.415169 / tr: 0.401599 (diff: 0.011687)
Fold  4 [  506] AUC : ho: 0.426898 / te: 0.415553 / tr: 0.402443 (diff: 0.011345)
Fold  5 [  421] AUC : ho: 0.426848 / te: 0.416714 / tr: 0.404175 (diff: 0.010134)
Fold  6 [  525] AUC : ho: 0.426496 / te: 0.416018 / tr: 0.401703 (diff: 0.010478)
Fold  7 [  366] AUC : ho: 0.427146 / te: 0.415603 / tr: 0.405945 (diff: 0.011543)
Full HO score 0.425851
FULL HO mean 0.426764, std 0.000220
FULL TE mean 0.415708, std 0.000487
FULL TR mean 0.403448, std 0.001474
FULL DIFF mean 0.011056, std 0.000536
Run LightG

Fold  4 [  419] AUC : ho: 0.426327 / te: 0.415431 / tr: 0.404654 (diff: 0.010896)
Fold  5 [  361] AUC : ho: 0.426848 / te: 0.416557 / tr: 0.406054 (diff: 0.010290)
Fold  6 [  490] AUC : ho: 0.427257 / te: 0.415969 / tr: 0.402856 (diff: 0.011288)
Fold  7 [  392] AUC : ho: 0.426867 / te: 0.415724 / tr: 0.405441 (diff: 0.011143)
Full HO score 0.425703
FULL HO mean 0.426611, std 0.000377
FULL TE mean 0.415701, std 0.000438
FULL TR mean 0.403982, std 0.001362
FULL DIFF mean 0.010910, std 0.000397
Run LightGBM with kfold - done in 133s
#### 69/71 NO DROP col: orderid_count_in_next_n_time_unit_6hour

start... 70/71 fe:model
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 72) (40024, 72) (40024, 72)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  426] AUC : ho: 0.427002 / te: 0.416106 / tr: 0.404852 (diff: 0.010896)
Fold  2 [  369] AUC : ho: 0.427430 / te: 0.415

In [9]:
report = pd.read_csv(report_path)
droped_col = list(report[report['drop']=='y'].feature.values)

In [10]:
saved_col = list(set(train_df.columns) - set(droped_col))
train_save = train[saved_col].copy()
print(train_save.shape)
test_save = test[saved_col].copy()
print(train_save.shape)
train_save.to_pickle('../../data/lgb_feature/train_lgb_agg2_drop_cv.pickle')
test_save.to_pickle('../../data/lgb_feature/test_lgb_agg2_drop_cv.pickle')

(1001650, 73)
(1001650, 73)


In [11]:
# #1 run base line
# report_path = 'report_lbfm.csv'
# target='click'
# train_index = pickle.load(open(FILE.train_index.value,'rb'))
# holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
# train_df = train.loc[train_index].copy()
# holdout = train.loc[holdout_index].copy()
# print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
# print('train mean: {}, holdout mean: {}'.format(train_df[target].mean(),holdout[target].mean()))
# try:
#     report = pd.read_csv(report_path)
#     print('found report! resume from last time!')
# except:
#     print('calculate importance to generate report...')
#     split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)
#     fea = [f for f in split_all.feature.values if f in search_feature]
#     report = pd.DataFrame({'feature':fea})
#     report['order'] = report.index
#     report['holdout'] = np.nan
#     report['holdout_mean'] = np.nan
#     report['search'] = 'n'
#     report['add'] = 'n'
#     report['cv'] = np.nan
# added = report[report['add']=='y'].copy()
# added_col = []
# if len(added) == 0:
#     print('calculate base line...')
#     _, _,holdout_base,_,cv_base,_,_,_ = runlgb(train_df[init_feature], 
#                                                test[init_feature], 
#                                                holdout[init_feature])
# else:
#     added = added.sort_values('order')
#     holdout_base = added.iloc[-1]['holdout']
#     cv_base = added.iloc[-1]['cv']
#     added_col = list(added['feature'].values)
# print('---- holdout base is {}. cv base is {}'.format(holdout_base,cv_base))
# print('added features are: {}'.format(added_col))
# best_col = init_feature + added_col

# searched = report[report['search']=='y'].copy()
# start_index = len(searched)


# for i in range(start_index,len(report)):
#     print('start... {}/{} fe:{}'.format(i,len(report),col))
#     col = report.iloc[i]['feature']
#     _, _,ho_cur,ho_mean_cur,cv_cur,_,_,_ = runlgb(train_df[best_col+[col]], 
#                                                test[best_col+[col]], 
#                                                holdout[best_col+[col]])
#     report.loc[i,'holdout'] = ho_cur
#     report.loc[i,'holdout_mean'] = ho_mean_cur
#     report.loc[i,'search'] = 'y'
#     report.loc[i,'cv'] = cv_cur
#     if cv_cur < cv_base and ho_cur < holdout_base:
#         cv_base = cv_cur
#         holdout_base = ho_cur
#         report.loc[i,'add'] = 'y'
#         print('#### {}/{} YES ADD col: {}'.format(i,len(report),col))
#         best_col = best_col + [col]
#     else:
#         report.loc[i,'add'] = 'n'
#         print('#### {}/{} NO ADD col: {}'.format(i,len(report),col))
#     report.to_csv(report_path,index=False)
#     print('')

    

    

In [12]:
## 24	osv_count_in_next_n_time_unit_6hour	24	0.426921	0.42762	y	y	0.416452